# *Archivo para probar los prompt*

## Instalaciones

In [37]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Obtener un paciente con su evolucion
Se hace una consulta a la BBDD para conseguir el historial de consultas de un paciente dado por su id de Paciente

Pacientes top 5 con mas consultas:
- id 172, Leo Martínez, consultas 3
- id 247, Luciana Gonzalez, consultas 3
- id 280, Matías Alejandro López Torres, consultas 3
- id 131, Lucas González, consultas 2
- id 153, Leonardo Martinez, consultas 2

In [ ]:
import sqlite3
import pandas as pd
import pprint

def query_to_database(query, db):
    # 1. Connect to the SQLite database
    conn = sqlite3.connect(db)

    try:
        df = pd.read_sql_query(query, conn)
        # print(df)
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()
        return df

# Obtener evolucion de un paciente
id_paciente = 172
sql_query_evolucion = f"""
SELECT * FROM evolucion WHERE id_paciente = {id_paciente};
"""
evolucion_df = query_to_database(sql_query_evolucion, '../01_setup/data/consultorio.db')

# Obtener datos personales del paciente
sql_query_paciente = f"""
SELECT * FROM paciente WHERE id_paciente = {id_paciente};
"""
paciente_df =query_to_database(sql_query_paciente, '../01_setup/data/consultorio.db')

# print("Datos paciente:")
# pprint.pprint(paciente_df) 
# print("Evolucion del paciente:")
# pprint.pprint(evolucion_df)


## Test de propt

### preparacion de datos de paciente y evoluciones

Formateo de clases

In [39]:
# configurar el parser de salida
from typing import Optional
from pydantic import BaseModel, Field

class Paciente(BaseModel):
    id_paciente: int = Field(...)
    apellido: str = Field(...)
    nombre: str = Field(...)
    fecha_nac: str = Field(...)
    sexo: str = Field(...)
    edad: str = Field(...)
    dni: str = Field(...)
    localidad: str = Field(...)
    obra_social: str = Field(...)
    afiliado_nro: str = Field(...)
    telefono: str = Field(...)
    telefono_numero: str = Field(...)
    email: str = Field(...)
    especialidad: str = Field(...)
    diagnostico: str = Field(...)
    enfermedad_base: str = Field(...)
    ant_perinatales: str = Field(...)
    ant_familiares: str = Field(...)
    registro: str = Field(...)
    fecha_registro: str = Field(...)
    
class Evolucion(BaseModel):
    id: int = Field(...)
    id_paciente: int = Field(...)
    fecha: str = Field(...)
    edad: int = Field(...)
    uni_edad: Optional[str] = Field(None)
    edad_anios: float = Field(...)
    edad_texto: str = Field(...)
    peso: float = Field(...)
    talla: float = Field(...)
    imc: float = Field(...)
    pc: float = Field(...)
    motivo: str = Field(...)
    conducta: Optional[str] = Field(None)

Creando objetos con los datos de paciente y evoluciones

In [40]:
paciente = Paciente(**paciente_df.to_dict(orient='records')[0])
evoluciones = [Evolucion(**item) for item in evolucion_df.to_dict(orient='records')]

# pprint.pprint(paciente)
# pprint.pprint(evoluciones)

### Preparacion de Prompt

In [ ]:
#%pip install langchain-openai
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    input=["paciente", "evoluciones"],
    template="""
    ### preparacion de datos de paciente y evoluciones
        Paciente: {paciente}
        Evoluciones: {evoluciones}
    ### Preparacion de Prompt
        Eres un asistente que ayuda a redactar notas medicas a partir de los datos del paciente y sus evoluciones. Tu tarea es generar un resumen de la historia clinica del paciente, incluyendo los datos relevantes de las evoluciones.
        Debes tener en cuenta:
            - que las evoluciones pueden tener diferentes formatos y contenidos, por lo que debes ser capaz de interpretar y sintetizar la informacion de manera coherente.
            - cruzar informacion de las diferentes evoluciones para identificar patrones, cambios y tendencias en la salud del paciente.
            - cruzar los antecendentes personales y familiares del paciente, asi como su diagnostico y enfermedad base.
    ### Salida esperada:
        Se espera que la salida sea un texto en pocos parrafos.
        El texto debe ser claro y conciso, utilizando lenguaje medico apropiado.
        Al comenzar la redaccion, incluye una breve introduccion con los datos personales del paciente (nombre, edad, diagnostico -si es que lo tiene-, enfermedad base -si es que lo tiene-).
        Cada patron, tendencia o cambio identificado debe ser explicado de manera clara y breve precisa en un parrafo aparte, utilizando como dato de referencia la fecha de la evolucion en la que se toma la informacion.
        No debes:
            - enumerar las evoluciones ni copiar literalmente su contenido.
            - incluir informacion sobre obras sociales ni numero de documento. 
            - confundir el termino "Control de salud" con diagnosticos ni enfermedades. simplemente obvia ese dato si es que aparece en las evoluciones.
            - confundir "Otra" con diagnosticos ni enfermedades. simplemente obvia ese dato si es que aparece en las evoluciones.
            - inventar informacion que no este presente en los datos del paciente y sus evoluciones.
    """,
    output_parser=StrOutputParser()
)

### Consulta al Modelo
preparando modelo

In [42]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

# Configurar la API key de OpenAI
openai_api_key = os.getenv("OPENAI_API_KEY")
llm_model = "gpt-4o-mini"
llm = ChatOpenAI(openai_api_key=openai_api_key, model = llm_model, temperature=0.7)

obteniendo resultados

In [43]:
chain = prompt | llm | StrOutputParser()
input_data = {
    "paciente": paciente,
    "evoluciones": evoluciones
}
respuesta = chain.invoke(input = input_data)
pprint.pprint(respuesta)

('**Historia Clínica del Paciente**\n'
 '\n'
 'El paciente, Leo Martínez, es un varón de 6 meses de edad. Se encuentra en '
 'seguimiento por control de salud, con antecedentes de control de salud '
 'regular. No presenta enfermedades base significativas, aunque se reporta un '
 'diagnóstico de "Control de salud". Entre sus antecedentes familiares, se '
 'destaca que la madre tiene un antecedente de anemia durante el embarazo, lo '
 'cual podría ser relevante para su seguimiento nutricional.\n'
 '\n'
 'A lo largo de las evoluciones, se ha observado un crecimiento constante en '
 'Leo. En la primera evaluación realizada el 9 de mayo de 2025, su peso era de '
 '7.8 kg y la talla de 65 cm, con un IMC de 18.46, indicando un estado '
 'nutricional adecuado para su edad. Se recomendó continuar con la '
 'suplementación de hierro y vitaminas, así como realizar un seguimiento de la '
 'ecografía de cadera debido a la falta de osificación del núcleo femoral, '
 'programada para el 11 de julio d

In [60]:
#Test de prompt 2

#%pip install langchain langchain-community openai

import sqlite3
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


DB_PATH = '../01_setup/data/consultorio.db'

def query_to_database(query: str, db: str) -> pd.DataFrame:
    conn = sqlite3.connect(db)
    try:
        df= pd.read_sql_query (query, conn)
    except sqlite3.Error as e:
        print ("Error al ejecutar= {e}")
        df = pd.DataFrame()
    finally:
        conn.close()
    return df

#Obtener todos los pacientes
sql_query_pacientes = "SELECT * FROM paciente;"
pacientes_df = query_to_database(sql_query_pacientes, DB_PATH)

#Obtener todas las evoluciones
sql_query_evoluciones = "SELECT * FROM evolucion;"
evoluciones_df = query_to_database(sql_query_evoluciones, DB_PATH)

#Debug de lectura de base de datos 
print ("Total de pacientes: ", len(pacientes_df))
print ("Total de evoluciones: ", len(evoluciones_df))

#Inicio de modelo GPT
llm = ChatOpenAI(model_name="gpt-4", temperature=0.3)
template="""
    ### Contexto
    Eres un médico analista encargado de elaborar un informe clínico.
    Tienes en memoria los pacientes: {pacientes}
    Y sus evoluciones: {evoluciones}

    ### Capacidades adicionales
    - Puedes realizar búsquedas en internet si lo consideras necesario para complementar el análisis.
    - Usa esta capacidad especialmente para proponer **posibles tratamientos o curas** basadas en información médica actualizada.

    ### Tarea
    1. Identifica al paciente que tenga la mayor cantidad de entradas en la tabla "evolucion".
    2. Redacta un análisis de su caso en varios párrafos claros y estructurados.
    3. El texto debe cumplir las siguientes reglas:
        - Comenzar con una introducción breve mencionando los estudios realizados.
        - Referirse siempre al paciente por su **nombre, apellido y edad**.
        - Elaborar un **diagnóstico** basado en la información disponible.
        - Describir las **posibles causas** de la condición observada.
        - Proponer **posibles tratamientos o curas**, usando tanto los datos del paciente como la información más actualizada disponible mediante búsquedas en internet.
        - Usar un lenguaje médico claro y conciso, en párrafos, evitando enumeraciones.
        - No inventar información que no esté presente en los datos del paciente o en la evidencia científica encontrada en internet.
"""

prompt = PromptTemplate(
    input_variables=["pacientes", "evoluciones", "instruccion"],
    template=template,
)

chain = LLMChain(llm = llm, prompt = prompt)

pacientes_sample = pacientes_df.head(15).to_dict(orient="records")
evoluciones_sample = evoluciones_df.head(20).to_dict(orient="records")
instruccion_usuario = "Hablar solo del paciente identificado por su nombre, no mencionar el ID"

respuesta = chain.run(
    pacientes=pacientes_sample,
    evoluciones=evoluciones_sample,
    instruccion=instruccion_usuario
)

print("=== Conclusión del modelo ===")
print(respuesta)


Total de pacientes:  70
Total de evoluciones:  79
=== Conclusión del modelo ===
    Tras revisar los registros de los pacientes y sus respectivas evoluciones, se identificó que el paciente con la mayor cantidad de entradas en la tabla "evolución" es Leo Martínez, de 6 meses de edad. 

    Leo Martínez, un varón de 6 meses de edad, ha sido sometido a varios controles de salud desde su nacimiento. Según los registros, Leo ha mantenido un peso y talla adecuados para su edad, con un índice de masa corporal (IMC) de 18.46 a los 6 meses. Además, se ha registrado un perímetro cefálico (PC) de 43.0 cm. En su última visita, se mencionó que continúa con la suplementación de sulfato ferroso 0.6 ml/día y vitaminas ADC 0.3 ml/día. También se mencionó que se realizará una ecografía de cadera a los 9 meses debido a la falta de osificación del núcleo femoral. 

    En base a la información disponible, Leo parece estar creciendo y desarrollándose adecuadamente para su edad. Sin embargo, la suplementaci